In [2]:
%matplotlib inline

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

メソッド`def forward`で、順伝搬関数を定義。
順伝搬とは、入力がNNに与えられ、それが順々に出力側に伝わっていき、最終的に損失関数の値まで計算が終わるまでの処理のこと。
今回は、ReLUとLinearを使って、順伝搬を定義している。

（C++クラスとの違い：pythonのクラスでのメンバ関数は、selfをつける。クラスの実装と定義を同じところに書く。）

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


上記の書き方で、インスタンス生成とともに`forward`関数と、`background operations`が実行される。
model.forward() と記載して入力データを処理してはいけない。

In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


In [11]:
input_image=torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten=nn.Flatten()
flat_image=flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
layer1=nn.Linear(in_features=28*28,out_features=20)
hidden1=layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1=nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1062, -0.0169, -0.2178, -0.4964,  0.0599, -0.2101, -0.1504,  0.3836,
         -0.2225, -0.2292,  0.3008,  0.0833,  0.5381,  0.5745, -0.1209, -0.2586,
         -0.1066,  0.0212, -0.2878,  0.4242],
        [ 0.7256,  0.1190, -0.1122, -0.2732,  0.0113, -0.0875, -0.0295,  0.2944,
         -0.1005, -0.2368,  0.5143,  0.0534,  0.2888,  0.7463,  0.1955, -0.2537,
         -0.0198, -0.1749, -0.1517,  0.2248],
        [ 0.3957,  0.2007, -0.3092,  0.0224,  0.3634, -0.3240, -0.3371,  0.2144,
         -0.3633, -0.0826,  0.1160, -0.1426,  0.5254,  0.0460, -0.1316, -0.0632,
          0.0734, -0.2812, -0.1032,  0.4298]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1062, 0.0000, 0.0000, 0.0000, 0.0599, 0.0000, 0.0000, 0.3836, 0.0000,
         0.0000, 0.3008, 0.0833, 0.5381, 0.5745, 0.0000, 0.0000, 0.0000, 0.0212,
         0.0000, 0.4242],
        [0.7256, 0.1190, 0.0000, 0.0000, 0.0113, 0.0000, 0.0000, 0.2944, 0.0000,
         0.0000, 0.5143, 0.0534, 0.2888, 0.7463, 0.19

ReLU: 非線形活性化関数の１つ。入力が0以下の場合は0を出力し、0より大きい場合はそのまま出力する。
nn.ReLU()(hidden1)で、インスタンス化したReLUをhidden1に適用している。

In [15]:
seq_modules=nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image=torch.rand(3,28,28)
logits=seq_modules(input_image)

In [16]:
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)

In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0127, -0.0260, -0.0065,  ...,  0.0050, -0.0170,  0.0348],
        [-0.0020, -0.0173, -0.0017,  ...,  0.0270,  0.0068, -0.0053]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0172, 0.0026], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0295,  0.0224,  0.0007,  ..., -0.0009, -0.0265, -0.0312],
        [ 0.0036,  0.0311,  0.0248,  ...,  0.0442,  0.0020,  0.0059]],
       device='cuda:0'